## The data in Earth engine can be exported to Google Cloud Storage as the TFRecord and GeoTiff format data

In [ ]:
## Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()
## Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()


In [19]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import folium
import time


In [28]:
# Your Earth Engine username.
User_Name = 'xin_luo'
# Ouput Cloud Storage bucket.
Output_Bucket = 'earth-engine-bucket-1'

# Select image data and the corresponding spectral bands.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
Export_Region = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])

# File names of the data colected from Earth Engine.
Image_File_Prefix = 'Landsat_img'
Image_File_Path = 'gs://' + Output_Bucket +'/' + Image_File_Prefix + '.tif'

### Selected the image from Earth Engine and visulization

In [ ]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).filterBounds(Export_Region).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[38., -122.5])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map


In [36]:
# Make sure you can see the output bucket.  You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + Output_Bucket) 
    else 'Can not find output Cloud Storage bucket.')

Found Cloud Storage bucket.


In [ ]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=Image_File_Prefix,
  bucket=Output_Bucket,
  scale=30,
#   fileFormat = 'TFRecord',
  fileFormat = 'GeoTIFF',
  region = Export_Region.toGeoJSON()['coordinates'],
#   formatOptions=image_export_options,   # Corresponding to the TFRecord export
)

# Start the task.
image_task.start()
while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  time.sleep(30)
print('Done with image export.')